# Yada Deconvolution

---



- Run the following cells for deconvolution using Yada.

## 1 - Import prerequisites

In [3]:
from IPython.display import FileLink, FileLinks
import pandas as pd
from yada import *

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

!pip install - q tslearn
!pip -q install gseapy
#!git clone https://github.com/zurkin1/Yada.git
#!mv Yada/* .

C:\Anaconda3\lib\site-packages\tslearn\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)
ERROR: Invalid requirement: '-'


## 2 - Configure Input Files.

Example input files are in the ./data/ folder.


In [4]:
#This is the mixture file in the format: columns: mix1, mix2, ..., rows: gene names.
mix = './data/Abbas/mix.csv'

#Reference matrix name. Should be normalized as the mix data.
pure = './data/Abbas/pure.csv'

#True cell type proportions file.
labels = './data/Abbas/labels.csv'

## 3 - Run deconvolution.

In [2]:
#Select deconvolution method. Available methods: run_dtw_deconv (recommended for Microarray), cibersort, dsection, lasso, nnls_deconv_constrained, pxcell (recommended for RNASeq).
result = run_dtw_deconv(mix, pure)
result

#Download Result.
#FileLink('data/results.csv')
#from google.colab import files
#files.download('data/results.csv') 

Deconvolution, num_cells: 4, num_mixes: 12
 99%

,Jurkat,IM-9,Raji,THP-1
0,0.608240,0.011822,0.097661,0.256301
1,0.580240,0.007622,0.100191,0.237226
2,0.638750,0.003526,0.123812,0.319146
3,0.179708,0.308647,0.420294,0.000144
4,0.182898,0.296129,0.548726,0.052655
5,0.179934,0.305920,0.492066,0.027743
6,0.028459,0.737727,0.013161,0.248349
7,0.026812,0.774451,0.002531,0.235900
8,0.028065,0.714551,0.023725,0.259095
9,0.003034,0.422357,0.290716,0.310899


## 4 - Evaluate Results.

In case true proportions are available.

In [3]:
calc_corr(labels, result)

Jurkat, 0.9932029300645067, SpearmanrResult(correlation=0.9716254134469436, pvalue=1.3811738967574605e-07)
IM-9, 0.990164022868434, SpearmanrResult(correlation=0.9716254134469436, pvalue=1.3811738967574605e-07)
Raji, 0.9857187838767321, SpearmanrResult(correlation=0.9716254134469436, pvalue=1.3811738967574605e-07)
THP-1, 0.945942905807847, SpearmanrResult(correlation=0.6909336273400487, pvalue=0.012843447163603845)


## 5 - Another Example With RNASeq.

Using xCell: ran, Hu and Butte, xCell: digitally portraying the tissue cellular heterogeneity landscape. Genome Biology (2017) 18:220.

In [5]:
#This is the mixture file in the format: columns: mix1, mix2, ..., rows: gene names.
mix = './data/Challenge/input/mix-107019.csv'

#True cell type proportions file.
#labels = './data/Abbas/labels.csv'

pxcell(mix)

Number of samples: 20, number of gene sets: 489


,mix0,mix1,mix2,mix3,mix4,mix5,mix6,mix7,mix8,mix9,mix10,mix11,mix12,mix13,mix14,mix15,mix16,mix17,mix18,mix19
cell_type,,,,,,,,,,,,,,,,,,,,
Adipocytes,0.000000,0.002355,0.000000,0.000104,0.000000,0.000000,0.000003,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003059,0.002760
Astrocytes,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
B-cells,0.060759,0.045040,0.053219,0.065533,0.000000,0.104229,0.000000,0.133439,0.080371,0.000947,0.111579,0.033880,0.000000,0.133060,0.112339,0.045254,0.058061,0.000000,0.000000,0.022430
Basophils,0.000000,0.000000,0.000000,0.025438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030487,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
CD4+ T-cells,0.000000,0.000000,0.064289,0.000000,0.000000,0.044092,0.000000,0.058988,0.047214,0.047096,0.000000,0.031886,0.000000,0.000000,0.008516,0.000024,0.102259,0.081592,0.001961,0.046897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ly Endothelial cells,0.000524,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000192,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
mv Endothelial cells,0.000188,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
naive B-cells,0.000000,0.002835,0.000000,0.022392,0.004026,0.021188,0.000000,0.000000,0.000000,0.000870,0.000000,0.011545,0.003523,0.012027,0.015073,0.000000,0.017573,0.003951,0.000000,0.006758
